The aim of this notebook is to show how you can go about analysing the performance of candlestick patterns. Here we introduce the concept of a win loss ratio, where the candlestick is considered predictive if it predicts the correct direction of stock prices significantly more than 50% of the time. This notebook is designed to be implemented in Quantopian's research environment. More commentary on the code can be found on AAAQuant's Candlestick Pattern Study blog at aaaquants.com.

In [61]:
import talib
import numpy as np
from matplotlib.pyplot import plot,show,subplots,legend
import pandas as pd
from quantopian.pipeline.factors.morningstar import MarketCap
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline
import seaborn as sns

In [62]:
def make_pipeline(x):
    '''Create pipeline based on top x market cap'''
    mktCap = MarketCap()
    myMktCap = mktCap.top(x)
    pipe = Pipeline()
    pipe.set_screen(myMktCap)
    
    return pipe

In [63]:
def pipeline_output(x, d1):
    '''Run the pipeline and let user specify x market cap at certain date'''
    pipe = make_pipeline(x)
    myPipe = run_pipeline(pipe, start_date=d1, end_date=d1)
    
    return myPipe

In [64]:
def get_stocks(x, d1):
    '''Create list of stocks from multi index data frame for future analysis'''
    
    #Call the pipeline_output function and specify the market cap and date
    #myPipe is a  multi index dataframe with date and stocks as the indices
    myPipe = pipeline_output(x, d1)
    
    #Count the multi index to check there are correct number of stocks for the date. 
    #Assert will break the code if the number of stocks is incorrect.
    assert(myPipe.reset_index().groupby('level_0')['level_1'].nunique()[0] == x)
    
    #Extract stocks from second level of index
    #Iterate through stocks and create new list with just the symbols as strings from the index
    myStocks = myPipe.index.get_level_values(1)
    myStocks = [str(i.symbol) for i in myStocks]
    
    return myStocks

In [65]:
def forward_returns(priceData, periodList, x):
    '''Calculates the periodList forward return e.g. 1min; x: how many stocks to go over'''
    '''Adds priceData dataframe columns named FwdReturn1, FwdReturn5 for 1 min and 5 min fwd returns'''
    for period in periodList:
    #shifted for correlation analysis with indicator signal so they are on the same row for analysis 
        priceData['FwdReturn%s'%period] = (priceData['price'].pct_change(period*x)*100).shift(-period*x)
    
    return priceData

In [66]:
def prep_price_rets(x, d1, d2, freq, periodList):
    '''Calling above functions'''
    '''Generates multi index dataframe with columns for OHLC and different forward returns together'''
    
    #Call the get_stocks function to get list of stocks and name this list for future reference
    myStocks = get_stocks(x, d1)
    
    #Use get_pricing to retrieve OHLC and volume data for each stock in myStocks
    #priceData has been converted from panel to a multi index data frame
    #fields are volume and price otherwise stocks that do not have OHLC data for the minute are excluded, ruining our forward return calculations.
    priceData = get_pricing(myStocks, d1, d2, frequency=freq, fields=['volume','price']).fillna(0).to_frame()
    assert(priceData.reset_index().groupby('major')['minor'].nunique()[0] == x)  #checking all stocks are there
    priceRets = forward_returns(priceData, periodList, x)
    
    return priceRets

In [67]:
def get_all_indicators(priceDf):
    a = []
    try:
        a.append(talib.CDL2CROWS(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDL3BLACKCROWS(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDL3INSIDE(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDL3LINESTRIKE(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDL3OUTSIDE(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDL3STARSINSOUTH(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDL3WHITESOLDIERS(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLABANDONEDBABY(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLADVANCEBLOCK(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLBELTHOLD(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLBREAKAWAY(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLCLOSINGMARUBOZU(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLCONCEALBABYSWALL(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLCOUNTERATTACK(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLDARKCLOUDCOVER(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLDOJI(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLDOJISTAR(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLDRAGONFLYDOJI(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLENGULFING(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLEVENINGDOJISTAR(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLEVENINGSTAR(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLGAPSIDESIDEWHITE(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLGRAVESTONEDOJI(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLHAMMER(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLHANGINGMAN(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLHARAMI(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLHARAMICROSS(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLHIGHWAVE(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLHIKKAKE(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLHIKKAKEMOD(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLHOMINGPIGEON(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLIDENTICAL3CROWS(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLINNECK(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLINVERTEDHAMMER(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLKICKING(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLKICKINGBYLENGTH(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLLADDERBOTTOM(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLLONGLEGGEDDOJI(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLLONGLINE(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLMARUBOZU(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLMATCHINGLOW(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLMATHOLD(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLMORNINGDOJISTAR(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLMORNINGSTAR(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLONNECK(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLPIERCING(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLRICKSHAWMAN(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLRISEFALL3METHODS(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLSEPARATINGLINES(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLSHOOTINGSTAR(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLSHORTLINE(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLSPINNINGTOP(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLSTALLEDPATTERN(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLSTICKSANDWICH(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLTAKURI(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLTASUKIGAP(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLTHRUSTING(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLTRISTAR(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLUNIQUE3RIVER(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLUPSIDEGAP2CROWS(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
        a.append(talib.CDLXSIDEGAP3METHODS(priceDf['open_price'],priceDf['high'],priceDf['low'],priceDf['price']))
    
    except:
        b = np.empty((len(priceDf),))
        b[:] = np.nan
        a.append(b)
        
    return np.array(a)

In [68]:
def prep_indicator(stockRets,INDICATOR):  #we will index reference the indicators
    inds = get_all_indicators(stockRets)
    ind = inds[INDICATOR]
    
    return ind

In [69]:
def initialise_signal_col(priceRets, INDICATOR,d1,d2,freq):
    '''Creates signal column starting with first stock'''
    
    stockRets = prep_stock_rets(priceRets,0,d1,d2,freq)
    ind = prep_indicator(stockRets,INDICATOR)
    stockRets['indicator'] = ind 
    priceRets['sig'] = stockRets['indicator']
    
    return priceRets
    

In [70]:
def prep_stock_rets(priceRets,i,d1,d2,freq):
    stockRets = get_pricing(priceRets.index.get_level_values(1)[i],d1,d2,frequency=freq)
    #since this does not have the minor axis of the equity name which is required to insert back into priceRets
    iterables = [stockRets.index, [priceRets.index.get_level_values(1)[i]]]
    new_index = pd.MultiIndex.from_product(iterables, names=['major', 'minor'])
    stockRets.index = new_index
    
    return stockRets

In [71]:
def new_signal_col(priceRets,x,INDICATOR,d1,d2,freq):
    '''Creates adjacent signal column for next stock, and then adds them to become one column'''
    
    for i in range(1,x):
        stockRets = prep_stock_rets(priceRets,i,d1,d2,freq)
        
        ind = prep_indicator(stockRets,INDICATOR)
        
        stockRets['indicator'] = ind 
        priceRets['sig1'] = stockRets['indicator']
        priceRets['sig'] = priceRets[['sig','sig1']].fillna(0).sum(axis=1)
        priceRets.drop('sig1',inplace=True,axis=1,errors='ignore')
    
    return priceRets

In [72]:
#Calling initalise signal and new signal column functions
#Returns priceRets a multi index dataframe with pricing data AND signal data 
def prep_signals(priceRets,x,INDICATOR,d1,d2,freq):
    '''Functions to create a signal column from above and normalises it'''
    
    priceRets = initialise_signal_col(priceRets,INDICATOR,d1,d2,freq)
    priceRets = new_signal_col(priceRets,x,INDICATOR,d1,d2,freq)
    priceRets['sig'] = priceRets['sig']/100
    
    return priceRets

In [73]:
def check_signal_direction(priceRets, sigList, period):
    '''Makes list of checking if signal direction matches fwd returns'''
    '''+1 means signal correlated with return, -1 means signal is not correlated'''
    sigList = [0 if sig == 0 else 1 if np.sign(sig) == np.sign(priceRets['FwdReturn%s'%period][i]) else -1 for i,sig in enumerate(priceRets['sig'])]
    
    priceRets['Match%s'%period] = sigList
    sigList = []
    
    return priceRets, sigList

In [74]:
def prep_win_loss(priceRets, periodList):
    '''Calling above function and returns priceRets df with signals'''
    sigList = [] 
    for period in periodList:
        priceRets, sigList = check_signal_direction(priceRets, sigList, period)
    return priceRets 

In [87]:
def calc_pnl(priceRets, periodList):
    '''Calculates pnl by multiplying the fwd return and the signal for each period in periodList'''
    for period in periodList:
        priceRets['PnL%s'%period] = priceRets['FwdReturn%s'%period]*priceRets['sig']
        
    return priceRets

In [88]:
def create_price_rets(x, d1, d2, freq, periodList, INDICATOR):
    '''Calls all above functions to form priceRets'''
    priceRets = prep_price_rets(x, d1, d2, freq, periodList)
    priceRets = prep_signals(priceRets,x,INDICATOR,d1,d2,freq)
    priceRets = calc_pnl(priceRets, periodList)
    priceRets = prep_win_loss(priceRets, periodList)
    
    return priceRets

In [89]:
def win_loss_ratio(priceRets,period):
    '''Calculates win loss ratio based on number of good matches over sum of good and bad matches'''
    goodMatch = float(len(priceRets[priceRets['Match%s'%period]==1]))
    badMatch = float(len(priceRets[priceRets['Match%s'%period]==-1]))
    
    if (goodMatch+badMatch) != 0.0:
        winLossRatio = goodMatch/(goodMatch+badMatch)
    else:
        winLossRatio = 0.0
    
    return winLossRatio

In [90]:
def all_win_loss_ratios(priceRets,periodList):
    '''Adds win loss ratios to list for each period in user specified period list'''
    '''Calls above function'''
    winLossRatios = []
    for period in periodList:
        winLossRatio = win_loss_ratio(priceRets,period)
        winLossRatios.append(winLossRatio)
        
    return winLossRatios

In [91]:
def scaled_win_loss_ratio(priceRets,period):
    '''Calculates win loss ratio but scaled by multiplying pnl'''
    '''We can make sure the returns made with correct predictions outweigh'''
    goodPnL = np.sum(priceRets[priceRets['Match%s'%period]==1]['PnL%s'%period])
    badPnL = abs(np.sum(priceRets[priceRets['Match%s'%period]==-1]['PnL%s'%period]))
    
    if (goodPnL+badPnL) != 0.0:
        scaledWinLossRatio = goodPnL/(goodPnL+badPnL)
    else:
        scaledWinLossRatio = 0.0
    
    return scaledWinLossRatio

In [92]:
def all_scaled_win_loss_ratios(priceRets,periodList):
    '''Adds scaled win loss ratios to list for each period in user specified period list'''
    scaledWinLossRatios = []
    for period in periodList:
        scaledWinLossRatio = scaled_win_loss_ratio(priceRets,period)
        scaledWinLossRatios.append(scaledWinLossRatio)
    
    return scaledWinLossRatios

In [93]:
def metrics(priceRets,periodList):
    winLossRatios = all_win_loss_ratios(priceRets,periodList)
    scaledWinLossRatios = all_scaled_win_loss_ratios(priceRets,periodList)
    
    return winLossRatios, scaledWinLossRatios

In [94]:
def index_names():
    indNames = []
    for indName in dir(talib):
        if 'CDL' in indName:
            indNames.append(indName)
    
    return indNames

In [95]:
def prep_heatmap(visualDf,myMetric,periodList,INDICATOR):
    '''Preparing visualisation dataframes'''
    
    indName = index_names()[INDICATOR]
    visualDf['%s'%indName] = pd.Series(myMetric,index=periodList)
    
    return visualDf

In [96]:
def plot_heatmap(heatmapDf,mid):
    subplots(figsize=(35,10))
    sns.heatmap(heatmapDf,center=mid,annot=True,cmap="coolwarm_r")

In [ ]:
#priceRets is the new multi index dataframe with columns for OHLC and different forward returns
#user specifies the number of x stocks 
x = 25
d1 = '2017-12-01'
d2 = '2017-12-23'
freq = 'minute'
periodList = [1,5,10]
visualDf1 = pd.DataFrame()
visualDf2 = pd.DataFrame()
visualDf3 = pd.DataFrame()

for INDICATOR in range(len(index_names())):

    priceRets = create_price_rets(x, d1, d2, freq, periodList, INDICATOR)
    winLossRatios, scaledWinLossRatios = metrics(priceRets,periodList)
    
    heatmapDf1 = prep_heatmap(visualDf1,winLossRatios,periodList,INDICATOR)
    heatmapDf2 = prep_heatmap(visualDf2,scaledWinLossRatios,periodList,INDICATOR)
    
plot_heatmap(heatmapDf1,0.5)
plot_heatmap(heatmapDf2,0.5)